# Example Pyllelic Use-Case Notebook

## Background

This notebook illustrates the import and use of `pyllelic` in a jupyter environment.

See https://github.com/Paradoxdruid/pyllelic for further details.

## Pre-setup

### Obtaining fastq data

We can download rrbs (reduced representation bisulfite sequencing) data from the Encode project:
http://hgdownload.cse.ucsc.edu/goldenPath/hg19/encodeDCC/wgEncodeHaibMethylRrbs/

Those files are in unaligned fastq format.  We will need to align these to a reference human genome.

### Aligning reads (using command line tools)

To align reads, we'll use bowtie2 and samtools.  **TODO:** Rewrite the below to use `process.py` commands.

First, we need to download a genomic index sequence: http://hgdownload.soe.ucsc.edu/goldenPath/hg19

Typical command:
```shell
bowtie2 -p 8 -x {bowtie_index_filename_without_suffix} -U {fastq_file_name} | samtools view -bS - > out.bam
```

Notes:
* p is number of processor cores, adjust for your system
* instead of `out.bam` use a filename that encodes cell-line and tissue.  Our convention is: `fh_CELLLINE_TISSUE.TERT.bam`

Then, we need to sort the resultant bam file.

Typical command:
```shell
samtools sort -o sorted.bam out.bam
```

Finally, we need to build an index file (**pyllelic** can also do this, if missing).

Typical command:
```shell
samtools index sorted.bam
```

Now, that sorted file (again, rename to capture cell-line and tissue info) is ready to be put in the `test` folder for analysis by pyllelic!

### Aligning reads (using process.py)

In [ ]:
# Processing imports
from pathlib import Path

In [ ]:
# Set up file paths
index = Path(
    "/home/andrew/allellic/hg19.p13.plusMT.no_alt_analysis_set//hg19.p13.plusMT.no_alt_analysis_set"
)
fastq = Path("/home/andrew/allellic/wgEncodeHaibMethylRrbsU87HaibRawDataRep1.fastq.gz")

**WARNING:** The next command is processor, RAM, and time intensive, and only needs to be run once!

In [ ]:
# pyllelic.bowtie2_fastq_to_bam(index=index, fastq=fastq)

Next, we need to sort and index the bam file using samtools functions.

In [ ]:
bamfile = Path("/home/andrew/allellic/wgEncodeHaibMethylRrbsU87HaibRawDataRep1.bam")
pyllelic.samtools_sort(bamfile)

In [ ]:
sorted_bam = Path("")
pyllelic.samtools_index(b)

## Set-up

In [ ]:
import pyllelic

In [ ]:
# set up your disk location:
# base_path should be the directory we'll do our work in
# make a sub-directory under base_path with a folder named "test"
# and put the .bam and .bai files in "test"

# pyllelic.set_up_env_variables(
#     base_path="/Users/abonham/documents/test_allelic/",
#     prom_file="TERT-promoter-genomic-sequence.txt",
#     prom_start="1293000",
#     prom_end="1296000",
#     chrom="5",
# )

pyllelic.set_up_env_variables(
    base_path="/home/andrew/allellic/",
    prom_file="TERT-promoter-genomic-sequence.txt",
    prom_start="1293000",
    prom_end="1296000",
    chrom="chr5",
)

## Main Parsing Functions

In [ ]:
files_set = pyllelic.make_list_of_bam_files()  # finds bam files

In [ ]:
# Uncomment for debugging:
files_set

In [ ]:
# index bam and creates bam_output folders/files
positions = pyllelic.index_and_fetch(files_set)

In [ ]:
# Uncomment for debugging:
positions

In [ ]:
# Only needs to be run once, generates static files
pyllelic.genome_parsing()

# Can also take sub-list of directories to process
# pyllelic.genome_parsing([pyllelic.config.bam_directory / "fh_BONHAM_TISSUE.TERT.bam"])

In [ ]:
cell_types = pyllelic.extract_cell_types(files_set)

In [ ]:
# Uncomment for debugging
cell_types

In [ ]:
# Set filename to whatever you want
df_list = pyllelic.run_quma_and_compile_list_of_df(
    cell_types, "test4.xlsx"
)  # to skip quma: , run_quma=False)

In [ ]:
# Uncomment for debugging
df_list.keys()

In [ ]:
means = pyllelic.process_means(df_list, positions, files_set)

In [ ]:
# Uncomment for debugging
means

In [ ]:
modes = pyllelic.process_modes(df_list, positions, files_set)

In [ ]:
# Uncomment for debugging
modes

In [ ]:
diff = pyllelic.find_diffs(means, modes)

In [ ]:
# Uncomment for debugging
diff

## Write Output to excel files

In [ ]:
# Set the filename to whatever you want
pyllelic.write_means_modes_diffs(means, modes, diff, "Test3")

## Visualizing Data

In [ ]:
final_data = pyllelic.pd.read_excel(
    pyllelic.config.base_directory.joinpath("Test3_diff.xlsx"),
    dtype=str,
    index_col=0,
)

In [ ]:
final_data

In [ ]:
individual_data = pyllelic.return_individual_data(df_list, positions, files_set)

In [ ]:
# Uncomment for debugging
individual_data

In [ ]:
pyllelic.histogram(individual_data, "SORTED", "1295680")

In [ ]:
pyllelic.histogram(individual_data, "SORTED", "1295903")

In [ ]:
final_data.loc["SORTED"]